In [1]:
# %%

from dataclasses import dataclass
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import logging
import logging.handlers
import sys
import os

import logging
import logging.handlers
import sys
import os

import time

start_time = time.time()



In [2]:


logfilename = 'skug_stats'
logfileformat = '%(relativeCreated)d - %(levelname)s - %(message)s'

# Rename old log files
if not logging.getLogger().handlers:

    if os.path.exists(logfilename + '.log'):
        #Rename the old log file, replacing the old one if it exists
        os.remove(f'{logfilename}_old.log') if os.path.exists(
            f'{logfilename}_old.log') else None
        os.rename(logfilename + '.log', f'{logfilename}_old.log')

    logfilehandler = logging.FileHandler(logfilename + '.log', mode='w')

    console = logging.StreamHandler(sys.stdout)
    console.setLevel(logging.INFO)
    consoleformat = '%(levelname)s - %(message)s'
    console.setFormatter(logging.Formatter(consoleformat))

    # Create root logger
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)

    # Add the console handler to the root logger
    logger.addHandler(console)
    # Add the file handler to the root logger
    logger.addHandler(logfilehandler)
    logger.handlers[1].setFormatter(logging.Formatter(logfileformat))
else:
    logger = logging.getLogger()

# ==================== #
logger.info('\n\n========== Starting skug_stats ==========\n\n')

# %%

# Open csvs from the current directory
characters_df = pd.read_csv('characters.csv')
frame_data = pd.read_csv('frame_data.csv')
move_aliases = pd.read_csv('move_aliases.csv')

# Remove spaces from column names
characters_df.columns = characters_df.columns.str.replace(' ', '')
frame_data.columns = frame_data.columns.str.replace(' ', '')
move_aliases.columns = move_aliases.columns.str.replace(' ', '')

logger.info('Loaded csvs')
logger.info(f'characters_df: {characters_df.columns}')
logger.info(f'frame_data: {frame_data.columns}')
logger.info(f'move_aliases: {move_aliases.columns}')



INFO - 

========== Starting skug_stats ==========


INFO - Loaded csvs
INFO - characters_df: Index(['Character', 'CharacterStart', 'Color'], dtype='object')
INFO - frame_data: Index(['Character', 'MoveName', 'AltNames', 'Guard', 'Properties', 'Damage',
       'Meter', 'OnHit', 'OnBlock', 'Startup', 'Active', 'Recovery', 'Hitstun',
       'Blockstun', 'Hitstop', 'OnPushblock', 'Footer', 'ThumbnailURL',
       'FooterURL'],
      dtype='object')
INFO - move_aliases: Index(['Key', 'Value'], dtype='object')


In [3]:



@dataclass
class Character:
    name: str
    shortened_names: list
    color: str
    moves: list
    movement_options: list
    cancel_chain: list


@dataclass
class Move:
    character: str
    name: str
    alt_names: list[str]|None
    guard: str|None
    properties: list | str | None
    damage: int | str | None
    meter_gain_loss: int | str | None
    on_hit: int | None
    on_block: int | None
    startup: int | None
    active: int | None
    recovery: int | None
    hitstun: int | None
    blockstun: int | None
    hitstop: int | None
    notes: str | None

    def __post_init__(self) -> None:
        self.repeats = 0
        self.hits = get_hits(self)


def get_hits(move: Move):

    logger.debug(
        f'===== Getting hits for move: {move.character} {move.name} =====')

    # Remove any newlines from the damage or move name
    move.name = move.name.replace('\n', '')
    move.damage = str(move.damage).replace('\n', '')

    # Replace '*' with 'x' in damage
    move.damage = move.damage.replace('*', 'x')

    # Replace → with -> in damage
    move.damage = move.damage.replace('→', '->')

    logger.debug(f'Original damage: \'{move.damage}\'')
    original_damage = move.damage
    damage = move.damage

    re_in_brackets: re.Pattern[str] = re.compile(r'\[.*?\]')
    re_in_paren: re.Pattern[str] = re.compile(r'\((.*?)\)')
    x_n_regex = re.compile(r'([\d]+[\s]*?)(x|\*)(\d+)')
    x_n_bracket_regex = re.compile(r'(\[[\d,\s]*?\]\s?)(x|\*)(\d+)')

    # Hits contains the damage and possible chip damage for each hit

    hits = {'damage': [], 'chip': []}
    hits_damage = []
    hits_chip = []
    stars_hits_damage = []
    stars_hits_chip = []
    x_n_matches = []

    if isinstance(damage, str) and damage != '-' and damage != 'nan':

        x_n_matches = x_n_regex.findall(damage)

        if x_n_matches:
            for x_n_match in x_n_matches:

                if x_n_match:

                    num_repeats = int(x_n_match[2])
                    damage_before_xn = x_n_match[0]

                    damage = expand_x_n(damage, damage_before_xn, num_repeats)
        else:
            x_n_brackets_matches = x_n_bracket_regex.findall(damage)

            if x_n_brackets_matches:
                for x_n_brackets_match in x_n_brackets_matches:
                    expanded_damage = []
                    logger.debug('Damage is in brackets')
                    damage = expand_x_n(damage, x_n_brackets_match[0],
                                        int(x_n_brackets_match[2]))



        if move.character == 'ANNIE':

            annie_stars = re_in_brackets.search(damage)
            if annie_stars:
                stars_chip = re_in_paren.search(annie_stars.group(0))
                if stars_chip:

                    # remove the chip damage from the damage string
                    damage = damage.replace(stars_chip.group(0), '')
                    stars_hits_chip = stars_chip.group(1).split(',')
                    logger.debug(f'stars chip: {stars_hits_chip}')
                annie_stars = re_in_brackets.search(damage)
                if annie_stars:
                    damage = damage.replace(annie_stars.group(0), '')
                    stars_hits_damage = annie_stars.group(0).replace(
                        '[', '').replace(']', '').split(',')
                    logger.debug(f'stars damage: {stars_hits_damage}')

        chip_damage = re_in_paren.search(damage)
        if chip_damage:
            # Log the values in parentheses and log them including the parentheses
            damage = damage.replace(chip_damage.group(0), '')
            hits_chip.extend(chip_damage.group(1).split(','))

        hits_damage.extend(damage.split(','))

        hits_lists = (hits_damage, hits_chip)
        if stars_hits_damage or stars_hits_chip:
            hits_lists = (hits_damage, hits_chip, stars_hits_damage,
                          stars_hits_chip)

        # Remove any empty strings from the lists
        for hits_list in hits_lists:
            while '' in hits_list:
                hits_list.remove('')
            #Remove strings that are just whitespace
            while ' ' in hits_list:
                hits_list.remove(' ')
            # Convert the damage values to ints
            for i, hit in enumerate(hits_list):
                # Remove whitespace from the start and end of the string
                space_search = re.findall(r'^\s*(.*?)\s*$', hit)
                if space_search:
                    hit = space_search[0]

            
                #convert to int
                if hit.isnumeric():
                    hits_list[i] = int(hit) # type: ignore

        hits['damage'] = hits_damage
        hits['chip'] = hits_chip
        if stars_hits_damage or stars_hits_chip:
            hits['stars_damage'] = stars_hits_damage
            hits['stars_chip'] = stars_hits_chip
        # Log the damage and chip damage for each hit, getting the values from the hits dict

    logger.debug(f'Hits: {hits}')

    return hits


def expand_x_n(parent_string: str, damage: str, n: int) -> str:
    """
    Expand a string with xN notation so that the instances of damage
    before xN are expanded in-place by N times, comma separated.
    Args:
        parent_string (str): The string to expand
        damage (str): The damage to expand
        n (int): The number of times to expand the damage
    """
    original_damage = damage
    expanded_damage = None
    if '[' in original_damage:
        damage = re.sub(r'\[|\]', '', original_damage).replace(' ', '')
        expanded_list = damage.split(',') * n
        expanded_damage = ','.join(expanded_list)
    else:
        expanded_damage = ','.join([damage] * n)

    expanded_parent_string = parent_string

    if original_damage in expanded_parent_string:
        original_damage += f'x{n}'
        expanded_parent_string = expanded_parent_string.replace(
            original_damage, expanded_damage)

    return expanded_parent_string




In [4]:


characters: dict = {}
moves: dict = {}
# Create character objects
for character in characters_df.iterrows():
    index = character[0]
    row: pd.Series = character[1]

    name: str = row['Character']
    shortened_names: list = row['CharacterStart'].split(' ')
    color: str = row['Color']
    # Create character object
    character = Character(name, shortened_names, color, [], [], [])
    characters[name] = character

# Get moves for each character
for move in frame_data.iterrows():
    move_obj = Move(
        character=move[1]['Character'],
        name=move[1]['MoveName'],
        # Check if string
        alt_names=move[1]['AltNames'],
        guard=move[1]['Guard'],
        properties=move[1]['Properties'].split(' '),
        damage=move[1]['Damage'],
        meter_gain_loss=move[1]['Meter'],
        on_hit=move[1]['OnHit'],
        on_block=move[1]['OnBlock'],
        startup=move[1]['Startup'],
        active=move[1]['Active'],
        recovery=move[1]['Recovery'],
        hitstun=move[1]['Hitstun'],
        blockstun=move[1]['Blockstun'],
        hitstop=move[1]['Hitstop'],
        notes=move[1]['Footer'])
    altered_name: str = move[1]['Character'] + '_' + move[1]['MoveName']
    # Remove spaces from move names
    altered_name = altered_name.replace(' ', '')
    moves[altered_name] = move_obj
